# Analíticas de Twitter - Competencia

#### Meses: Marzo, Abril y Mayo 

#### Se necesita:
- Número de likes por post 
- Número de respuestas por post 
- Número de RTs por post 
- Número de seguidores 

#### Se desea:
- Número de post publicados en cada mes
- Interacción promedio por post (Likes + respuestas + RTs / Número de post publicados durante los 3 meses)

## Importamos las librerías necesarias

In [1]:
from tweepy import OAuthHandler
from tweepy import API
from tweepy import Cursor
import tweepy
import logging
from datetime import datetime, date, time, timedelta
from collections import Counter
import sys
import pandas as pd

## Inicializamos la instancia de la librería de scraping

In [2]:
consumer_key = "HoZXZP6eW5CW86V9EcEw4oBcP"
consumer_secret = "JKvzWsuXWsgDuAFeBW7i5MVLyDZ60AhXwSD0slBTQ70SX8Byga"
access_token = "285395514-ZzvKkqW5fJrolpwqyRiNPDtaQ43R517c050APBQ0"
access_token_secret = "m6ItBTrTV5SHWwXPSpDujWOpILIARaijHrtixxKWsHRKn"

In [3]:
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
auth_api = API(auth, wait_on_rate_limit=True)

In [4]:
def get_mean(sum, count):
    return sum / count if count != 0 else 0

In [5]:
def get_mean_daily_tweets(info):
    delta = datetime.utcnow() - info.created_at
    account_age_days = delta.days
    return 0 if account_age_days == 0 else info.statuses_count / account_age_days

## Definimos las cuentas a revisar

In [10]:
# accounts_list = ["NestleContigo ","AlimentosMaryVe","EmpresasPolar","Natulacoficial","Yukeryve ","EslandesOficial ","Lacteos_Yaracuy","lapastorena_ve","QuesosTorondoy","plumrosevzla","HermoVenezuela","EmbutidosRicci","charcuteria_ve","lamontserratina","Charvenca","Agronivar_tw","SACarabobo","HMotoresNag","purolomoalibal","FSantaTeresa","fundaciondusa","FUNDACIONBENGOA","FundacionNadbio"]
accounts_list = ['adrianaszb']

In [11]:
accounts_info = []
account_counter = 1
for target in accounts_list:
    # Basic info
    print(f"Getting data for {target}. {account_counter/len(accounts_list) * 100}%")
    api_response = auth_api.get_user(target)
    tweets_count = api_response.statuses_count
    account_info = dict()
    account_info['username'] = target
    account_info['created_at'] = api_response.created_at
    account_info['tweets_count'] = tweets_count
    account_info['follower_count'] = api_response.followers_count
    account_info['mean_daily_tweets'] = get_mean_daily_tweets(api_response)
    account_info['march_posts_count'] = 0
    account_info['april_posts_count'] = 0
    account_info['may_posts_count'] = 0
    # Extra info
    hashtags = []
    mentions = []
    tweet_count = 0
    retweet_sum = 0
    favorite_sum = 0
    end_date = datetime.utcnow() - timedelta(days=108)
    for status in Cursor(auth_api.user_timeline, id=target, q="filter:retweets").items():
        if(status.created_at.year != 2020):
            print("No tweets this year")
            break
        # Skip Retweets
        if status.text.startswith("RT @"):
            continue
#         Skip june tweets
        if status.created_at.month == 6:
            continue
        # Add to the month count
        if status.created_at.month == 5:
            account_info['may_posts_count'] += 1
        if status.created_at.month == 4:
            account_info['april_posts_count'] += 1
        if status.created_at.month == 3:
            account_info['march_posts_count'] += 1
        tweet_count += 1
        retweet_sum += status.retweet_count
        favorite_sum += status.favorite_count
        if hasattr(status, "entities"):
            entities = status.entities
            if "hashtags" in entities:
                for ent in entities["hashtags"]:
                    if ent is not None:
                        if "text" in ent:
                            hashtag = ent["text"]
                            if hashtag is not None:
                                hashtags.append(hashtag)
            if "user_mentions" in entities:
                for ent in entities["user_mentions"]:
                    if ent is not None:
                        if "screen_name" in ent:
                            name = ent["screen_name"]
                            if name is not None:
                                mentions.append(name)
        
        # Stop getting tweets after the end date
        if status.created_at < end_date:
            break
    account_info['most_used_hashtags'] = ', '.join(list(map(lambda item: item[0], Counter(hashtags).most_common(10))))
    account_info['most_mentioned_user'] = ', '.join(list(map(lambda item: item[0], Counter(mentions).most_common(10))))
    account_info['likes_sum'] = favorite_sum
    account_info['retweets_sum'] = retweet_sum
    account_info['mean_likes'] = get_mean(account_info['likes_sum'], tweets_count)
    account_info['mean_retweets'] = get_mean(account_info['retweets_sum'], tweets_count)
    account_info['mean_interaction'] = account_info['mean_likes'] + account_info['mean_retweets']
    accounts_info.append(account_info)
    account_counter += 1
print("Done")

Getting data for adrianaszb. 100.0%
Done


## Pasamos la información a un Pandas DataFrame

In [12]:
df = pd.DataFrame()
keys = ["username", "created_at", "tweets_count", "mean_daily_tweets", "retweets_sum", "likes_sum", 'follower_count', 'march_posts_count', 'april_posts_count', 'may_posts_count', 'mean_likes', 'mean_retweets', 'mean_interaction', "most_used_hashtags", "most_mentioned_user"]
for key in keys:
    df[key] = pd.Series(list(map(lambda info: info[key] if key in info.keys() else 0, accounts_info)))

## Exportamos a un CSV

In [13]:
df.to_csv('xd.csv')